In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [4]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# https://medium.com/towards-data-science/how-to-automate-3d-point-cloud-segmentation-and-clustering-with-python-343c9039e4f5

In [5]:
#create paths and load data
import open3d as o3d
input_path=".\\data\\"
dataname="TLS_kitchen.ply"

pcd = o3d.io.read_point_cloud(input_path+dataname)

In [6]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
pcd.paint_uniform_color([0.6, 0.6, 0.6])


PointCloud with 511026 points.

In [7]:
o3d.visualization.draw_geometries([pcd]) #Works only outside Jupyter/Colab

In [8]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

Plane equation: 0.00x + 0.00y + 1.00z + -0.96 = 0


PointCloud with 380203 points.

In [9]:
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [10]:
dataname="TLS_kitchen_sample.ply" #planes removed
pcd = o3d.io.read_point_cloud(input_path+dataname)

labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

point cloud has 13 clusters


In [11]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 


In [12]:
#Display one segment
#segment0 = pcd.select_by_index(list(np.where(labels == 0)[0]))
#segment0.paint_uniform_color([0,0,0])
#o3d.visualization.draw_geometries([segment0])


In [13]:
dataname="TLS_kitchen.ply" #planes removed
pcd = o3d.io.read_point_cloud(input_path+dataname)

segment_models={}
segments={}
max_plane_idx=10
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [14]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [15]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])



point cloud has 61 clusters


In [16]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=0.3199,front=[0.30159062875123849, 0.94077325609922868, 0.15488309545553303],lookat=[-3.9559999108314514, -0.055000066757202148, -0.27599999308586121],up=[-0.044411423633999815, -0.138726419067636, 0.98753122516983349])
o3d.visualization.draw_geometries([rest])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
